In [76]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [77]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [90]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')
loansContract = web3.eth.contract(address=loansContractAddress, abi=loansContractABI)

In [91]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0xc80459506f27992076e403e785b4b46a598328a55accc64aa8b7e3b22bfffcc3')

In [92]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0xb2e3dca17b6225f68683454cdf0601d3fd232f22e89478eeb07bdb4ea6e0059c')

In [81]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract, _installmentsNum ,  _interest):
    
    transaction = _organizationContract.functions.createLoan(_loanie, _amount, _installmentsNum ,  _interest
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    # _organizationContract.functions.createLoan(_loanie, _loaner, _amount).transact()
    return True

In [99]:
deleteUser()

In [97]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [98]:
# Suppose bank wants to create a loan
#loaner = input("Enter the loaner addrses: ")
loaner = web3.eth.accounts[5]
loaner
### Check to see whether it is a user or organization address
loanerIndex = accountsConract.functions.getIndex(loaner).call()

In [99]:
#loanie = input("Enter the loanie address: ")
loanie = web3.eth.accounts[2]
loanie
loanieIndex = accountsConract.functions.getIndex(loanie).call()

## Get pending loans

In [100]:
# Get pending loans using 3 separate functions
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [101]:
getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)

Enter your password: ········


['554858 0xd499503750a54D4bd6fA54C39991F8d5040ec781 1589722454 9 15 ',
 '873893 0xd499503750a54D4bd6fA54C39991F8d5040ec781 1589722458 10 6 ',
 '242362 0xd499503750a54D4bd6fA54C39991F8d5040ec781 1589722461 10 29 ']

## Confirm or reject loans

In [102]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for id: ' + pendingLoan.split(' ')[0])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 900000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 300000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your password: ········
Enter your loanie password: ········
for id: 554858
c/r?c
for id: 873893
c/r?c
for id: 242362
c/r?c


# Create a new loan

In [96]:
loaner
# Create a new loan
if loanerIndex != -1:
    loanerType = accountsConract.functions.getType(loanerIndex).call()
    if loanerType:
        # We will change this to inputs later
        amount=random.randint(1000,1000000)
        installmentsNum=random.randint(6,15)
        interest=random.randint(5,35)
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=loaner, _amount=amount, _organizationContract=organizationContract, _installmentsNum=installmentsNum, _interest=interest):
            print("Loan created")
else:
    print("This account is not registered in our system.")

Enter the password of the organization: ········
Loan created


## Get user loans

In [20]:
def getLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions. getMyLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [106]:
def getLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    loans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                loans.append(string)
    else:
        print("This account is not registered in our system.")
    return loans

In [107]:
getLoansList(loanieIndex,userContract,accountsConract,loanie)

Enter your password: ········


['554858 0xd499503750a54D4bd6fA54C39991F8d5040ec781 1589722454 9 15 ',
 '873893 0xd499503750a54D4bd6fA54C39991F8d5040ec781 1589722458 10 6 ',
 '242362 0xd499503750a54D4bd6fA54C39991F8d5040ec781 1589722461 10 29 ']

In [108]:
myId=1589717624
private_key = input("enter the private Key")
transaction = loansContract.functions.getInstallments(
int(myId)).buildTransaction({
'gas': 300000,
'gasPrice': web3.toWei('1', 'gwei'),
'from': loanie,
'nonce': web3.eth.getTransactionCount(loanie)
}) 
signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
receipt = web3.eth.getTransactionReceipt(transaction_hash)
rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
event_values = rich_logs[0]['args'] # Dictionary
print(event_values)

enter the private Keysss


ValueError: when sending a str, it must be a hex string. Got: 'sss'

In [114]:
def getInstallments(_userContract, _loanieAddress, _privateKey):
    myId = 1589722458
    transaction = _userContract.functions.getMyInstallments(myId
    ).buildTransaction({
    'gas': 300000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getInstallments().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values

In [115]:
def getinstallmentsList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getInstallments(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amount'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [116]:
getinstallmentsList(loanieIndex,userContract,accountsConract,loanie)

Enter your password: ········


['1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ',
 '1000 1589722550 0 False ']